Time Series

Para este proceso hacemos una asuncón:
los procesos que están bien se comportan de manera similar.

(FASE A)
- Concatenar (uno debajo del otro) el 10% de los df buenos
- Estandarizar los datos (StandardScaler)
- Entrenar pca

(FASE B)
- Con todos los df buenos, estandarizar y hacer PCA con los modelos
entrenados en la fase A.
- Calcular distancias (t2, Q o error de reconstrucción) de cada df bueno. Si la distancia es
grande, mal asunto NO SEGUIR.
- Definir threshold de para detectar si es error o no.


In [1]:
import os
import numpy as np
import pandas as pd
from glob import glob

files = glob('Z:\Descargas\CSV_DATA\*.csv')
col_names = {'Flow rate', 'Zone9_Pressure', 'Zone16_Pressure', 'Zone17_Pressure', 'Zone24_Pressure', 'TOTAL_QUALITY'}
target_name = 'TOTAL_QUALITY'

df_list = []
np.random.shuffle(files)

for file in files:
    df = pd.read_csv(file)[col_names]
    target = df[target_name]
    df = df.drop(target_name, axis=1)
    df[target_name] = target
    if df[target_name][0] == 1:
        df_list.append(df)
    if len(df_list) == 1000:
        break
        
print(len(df_list))

1000


In [2]:
df = pd.concat(df_list)
df = df.drop(target_name, axis=1)
df

,Zone24_Pressure,Zone16_Pressure,Zone17_Pressure,Flow rate,Zone9_Pressure
0,0.0,0.0,0.0,6.727330e-07,0.0
1,0.0,0.0,0.0,7.170040e-07,0.0
2,0.0,0.0,0.0,7.614100e-07,0.0
3,0.0,0.0,0.0,8.404510e-07,0.0
4,0.0,0.0,0.0,8.817000e-07,0.0
...,...,...,...,...,...
901,21197.8,22425.1,46659.5,9.772530e-80,63291.9
902,21780.2,22859.9,46793.7,1.009130e-81,63368.7
903,22553.2,23431.4,46974.4,9.631080e-84,63470.1
904,23169.3,23893.6,47113.6,7.996520e-86,63553.4


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(df)
df_scaled = scaler.transform(df)

In [4]:
from sklearn.decomposition import PCA
from models.model import ReconstructionErrorModel

pca = ReconstructionErrorModel(df_scaled, model=PCA(n_components=0.9))

train_errors = pca.predict(df_scaled)
threshold = np.mean(train_errors) + (np.std(train_errors) * 3)

# Print principal components
print(threshold)
print('%i Principal Components' % len(pca.get_model().explained_variance_))

0.3187726103168989
3 Principal Components


In [5]:
correct_list = []
incorrect_list = []

files = glob('Z:\Descargas\CSV_DATA\*.csv')
col_names = {'Flow rate', 'Zone9_Pressure', 'Zone16_Pressure', 'Zone17_Pressure', 'Zone24_Pressure', 'TOTAL_QUALITY'}
target_name = 'TOTAL_QUALITY' 
for file in files:
    df = pd.read_csv(file)[col_names]
    target_col = df[target_name]
    df = df.drop(target_name, axis=1)
    df[target_name] = target_col
    if df[target_name][0] == 1:
        correct_list.append(df)
    else:
        incorrect_list.append(df)

In [6]:
correct_list[0]

,Zone24_Pressure,Zone16_Pressure,Zone17_Pressure,Flow rate,Zone9_Pressure,TOTAL_QUALITY
0,0.0,0.0,0.0,6.727330e-07,0.0,1
1,0.0,0.0,0.0,7.170040e-07,0.0,1
2,0.0,0.0,0.0,7.614100e-07,0.0,1
3,0.0,0.0,0.0,8.404510e-07,0.0,1
4,0.0,0.0,0.0,8.817000e-07,0.0,1
...,...,...,...,...,...,...
852,14804.8,14720.4,66607.3,3.855150e-11,59676.3,1
853,15880.0,15792.2,66752.0,5.818180e-12,59802.5,1
854,17785.9,17987.0,67040.7,5.178050e-13,60052.4,1
855,19790.3,20133.1,67332.0,3.223350e-14,60308.1,1


In [9]:
anomalies = 0
for df in correct_list:
    df_input = df.drop(target_name, axis=1)
    df_input = scaler.transform(df_input)
    errors = pca.predict(df_input)
    errors = np.mean(errors)

    if errors > threshold:
        anomalies += 1
        
print('', str(anomalies), '/', str(len(correct_list)))

 356 / 5055


In [10]:
anomalies = 0
for df in incorrect_list:
    df_input = df.drop(target_name, axis=1)
    df_input = scaler.transform(df_input)
    errors = pca.predict(df_input)
    errors = np.mean(errors)
    if errors > threshold:
        anomalies += 1
        
print('', str(anomalies), '/', str(len(incorrect_list)))

 831 / 4959
